In [46]:
!pip install unsloth dagshub mlflow evaluate rouge_score bert_score -q --no-cache-dir

## Loading PLLuM model

In [1]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

pllum_model_name = "CYFRAGOVPL/Llama-PLLuM-8B-instruct"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = pllum_model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
import pandas as pd
from tqdm.notebook import tqdm
from informal_to_formal.evaluation import Evaluator


alpaca_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Translate informal text in Polish language to formal language.

### Input:
{}

### Response:
{}"""

alpaca_prompt_pl = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Przetłumacz poniższy nieformalny tekst w języku polskim na język formalny.

### Input:
{}

### Response:
{}"""


def generate_language_model_output(
        input: str,
        model,
        tokenizer,
        alpaca_prompt,
        temperature=0.3
    ) -> str:
    # Format the prompt using the provided input range
    FastLanguageModel.for_inference(model)

    inputs = tokenizer(
    [
        alpaca_prompt.format(
            input, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, use_cache=True, temperature=temperature)
    decoded_outputs = tokenizer.batch_decode(outputs)
    decoded_outputs = decoded_outputs[-1].split(
        "### Response:\n"
    )[-1].replace("<|end_of_text|>", "")

    return decoded_outputs


def generate_language_model_batch_output(
        input: list[str],
        model,
        tokenizer,
        alpaca_prompt,
        temperature=0.3,
        batch_size=16
    ) -> list[str]:
    # Format the prompt using the provided input range
    FastLanguageModel.for_inference(model)
    model.eval()

    all_outputs = []

    for i in tqdm(range(0, len(input), batch_size)):
        batch_input = input[i: i + batch_size]

        # Format prompts
        prompts = [
            alpaca_prompt.format(input_text, "")
            for input_text in batch_input
        ]
        inputs = tokenizer(prompts, padding=True, return_tensors = "pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                use_cache=True,
                temperature=temperature
            )
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        all_outputs.extend(decoded_outputs)

    # postprocess format
    all_outputs = [
        output.split("### Response:\n")[-1].replace("<|end_of_text|>", "")
        for output in all_outputs
    ]

    return all_outputs


def evaluate_language_model(
        pred: list[str], target: list[str]
    ) -> tuple[pd.DataFrame, dict]:
    evaluator = Evaluator(pd.DataFrame({
        "pred": pred,
        "target": target,
    }))
    evaluate_df_metrics, avg_metrics = evaluator.evaluate()
    return evaluate_df_metrics, avg_metrics

## Loading and processing the dataset

In [4]:
from dagshub.data_engine import datasources
import pandas as pd

ds = datasources.get("informal2formal/mlflow", "synthetic_data_source")
ds.head().dataframe

Accessing as directtt

Output()

,path,datapoint_id,dagshub_download_url,media type,size
0,informal_formal_synthetic_v1.csv,86087877,https://dagshub.com/api/v1/repos/informal2form...,text/plain,1221781


In [5]:
dataset_uri = ds.head().dataframe["dagshub_download_url"].values[0]
df = pd.read_csv(dataset_uri)
df.head()

Output()

,zdanie_nieformalne,zdanie_formalne,model,generator,split
0,"Nie cierpię, jak wchodzę do przymierzalni w sk...","Odczuwam dyskomfort, gdy wchodząc do przymierz...",gemini-2.5,seba,train
1,"Trzeba zakasać rękawy i zapierdalać, jeśli chc...",Konieczne jest wzmożenie wysiłków i intensyfik...,gemini-2.5,jedrek,train
2,Muszę się w końcu zabrać za naukę obsługi jaki...,Postanowiłem rozpocząć naukę obsługi jednego z...,gemini-2.5,seba,train
3,Weź się w końcu ogarnij z tym swoim ciągłym kr...,Apeluję o zmianę Pana/Pani postawy i zaprzesta...,gemini-2.5,jedrek,train
4,"Wpadnij do mnie na chwilę po pracy, musimy prz...",Zapraszam do siebie na chwilę po zakończeniu p...,gemini-2.5,jedrek,train


In [6]:
df_train = df[df['split'] == 'train'][['zdanie_nieformalne', 'zdanie_formalne']]
df_val = df[df['split'] == 'val'][['zdanie_nieformalne', 'zdanie_formalne']]
df_test = df[df['split'] == 'test'][['zdanie_nieformalne', 'zdanie_formalne']]

In [7]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

In [8]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["zdanie_nieformalne"]
    outputs      = examples["zdanie_formalne"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = alpaca_prompt_pl.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [9]:
train_dataset = train_dataset.map(
    formatting_prompts_func,
    batched=True,
)
val_dataset = val_dataset.map(
    formatting_prompts_func,
    batched=True,
)
test_dataset = test_dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/2450 [00:00<?, ? examples/s]

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

## Evaluate the baseline model

### Example inputs

In [10]:
example_inputs = [
    "Mógłbyś mi przypomnieć, jaki był deadline na oddanie tego raportu? Coś mi świta, że to było w tym tygodniu.",
    "Trzeba opierdolić tych z supportu, bo czekam na odpowiedź już trzeci dzień, a problem dalej nierozwiązany.",
    "Pojebało cię człowieku żeby tam iść samemu!?",
    "Ta cała moda na 'self-care' i 'dbanie o siebie' jest ważna, ale czasem mam wrażenie, że przeradza się w egoizm i usprawiedliwienie dla olewania obowiązków albo innych ludzi. Granica jest cienka.",
    "Ten nowy pracownik w zespole to jakiś leser, ciągle łazi na fajkę albo scrolluje TikToka zamiast coś robić."
]
base_model_outputs = []

for input in tqdm(example_inputs):
    generated_output = generate_language_model_output(input, model, tokenizer, alpaca_prompt_pl)
    base_model_outputs.append(generated_output)

    print(generated_output, "\n")

  0%|          | 0/5 [00:00<?, ?it/s]

Przypominam, że termin złożenia raportu upływał w tym tygodniu. 

Należy wystosować do supportu stanowczy monit w sprawie przyspieszenia procedowania zgłoszonego wniosku, ponieważ od momentu złożenia wniosek nie został jeszcze rozpatrzony. 

Niech Pan wybaczy, ale nie rozumiem powodu Pana oburzenia. 

Przedstawiamy tłumaczenie na język formalny:

Moda na "samopielęgnację" i "dbanie o siebie", która obecnie panuje, jest niewątpliwie istotna, jednakże niekiedy odnoszę wrażenie, że przekształca się ona w egoizm oraz usprawiedliwienie dla zaniedbywania obowiązków bądź innych osób. Granica jest bardzo cienka. 

Należy stwierdzić, że nowy pracownik nie wywiązuje się należycie z powierzonych mu obowiązków, gdyż zamiast wykonywać powierzone mu zadania, zajmuje się niepotrzebnymi czynnościami, takimi jak palenie papierosów czy korzystanie z portali społecznościowych. 



### Test dataset using `Evaluator`

In [11]:
# limit due to Collab resources & time restrictions
TEST_DATA_LIMIT=150
test_list_pred = df_test['zdanie_nieformalne'].tolist()[:TEST_DATA_LIMIT]
test_list_target = df_test['zdanie_formalne'].tolist()[:TEST_DATA_LIMIT]

# OOM for batch predictions, insufficent Collab resources
# base_model_predictions = generate_language_model_batch_output(test_list, model, tokenizer, alpaca_prompt_pl, batch_size=4)

base_model_predictions = [
    generate_language_model_output(input, model, tokenizer, alpaca_prompt_pl)
    for input in tqdm(test_list_pred)
]

  0%|          | 0/150 [00:00<?, ?it/s]

In [29]:
evaluate_df_metrics, avg_metrics = evaluate_language_model(
    base_model_predictions, test_list_target
)
avg_metrics

Computing ROUGE scores:   0%|          | 0/150 [00:00<?, ?it/s]

Computing BERT scores:   0%|          | 0/150 [00:00<?, ?it/s]

{'rouge1': 0.2918395075454998,
 'rouge2': 0.11858580673176443,
 'rougeL': 0.2395531189651921,
 'bert_precision': 0.7360879790782928,
 'bert_recall': 0.7345356667041778,
 'bert_f1': 0.7346676687399546}

In [33]:
evaluate_df_metrics.sort_values(by='bert_f1')

,pred,target,rouge1,rouge2,rougeL,bert_precision,bert_recall,bert_f1
4,Należy przetłumaczyć ten tekst na język formalny.,"Nowy program telewizyjny, w którym celebryci p...",0.000000,0.000000,0.000000,0.642632,0.572537,0.605563
48,"Chętnie przyrządzę dla Pana/Pani danie, które ...","Odczuwam silną ochotę na tradycyjną, domową zu...",0.000000,0.000000,0.000000,0.640927,0.586733,0.612634
131,"Dziękuję za informację, że udało się zrealizow...","Świetnie, gratuluję sukcesu!",0.000000,0.000000,0.000000,0.605877,0.622431,0.614043
143,Pragnę podziękować za informację o powodzeniu ...,"Wspaniale, że odnieśliśmy sukces!",0.000000,0.000000,0.000000,0.609888,0.632714,0.621092
77,Oto tekst w języku formalnym:\n\nBardzo przykr...,Automatyczne odpowiedzi mailowe informujące o ...,0.128205,0.000000,0.051282,0.620568,0.632604,0.626528
...,...,...,...,...,...,...,...,...
33,"Należy podkreślić, że integracyjny wyjazd był ...",Ten wyjazd integracyjny był bardzo udany.,0.625000,0.285714,0.500000,0.783073,0.885874,0.831307
120,Zastosowanie nowej funkcji w aplikacji do zama...,Nowa funkcja w aplikacji mobilnej do zamawiani...,0.444444,0.253165,0.444444,0.835129,0.846451,0.840752
86,Należy dopisać pokrycie testami w wysokości 60%.,Pokrycie kodu testami wynosi 60%. Należy dopis...,0.600000,0.222222,0.300000,0.876398,0.844137,0.859965
45,"Funkcja ""SharePlay"", dostępna w smartfonie iPh...",Nowa funkcja 'SharePlay' dostępna w systemie i...,0.637363,0.449438,0.549451,0.887097,0.846241,0.866188


## Model training

In [34]:
import dagshub

dagshub.init(repo_owner="informal2formal", repo_name="mlflow", mlflow=True)

Initialized MLflow to track repo "informal2formal/mlflow"

Repository informal2formal/mlflow initialized!

In [35]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",

    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

run_name = "PLLuM-1-full-epoch"

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 15,
        num_train_epochs = 1,  # max_steps = 60,
        eval_strategy="steps", # val_strategy="epoch",
        eval_steps=100, # eval_steps=0.33,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = run_name,
        report_to="mlflow",
    )
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2450 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/350 [00:00<?, ? examples/s]

In [36]:
import mlflow

mlflow.set_experiment("PLLuM unsloth.ai")

with mlflow.start_run(run_name=run_name):
    trainer_stats = trainer.train()

    mlflow.log_param("base_model_name", pllum_model_name)
    mlflow.log_param("prompt_template", alpaca_prompt_pl)
    mlflow.log_param("dataset_uri", dataset_uri)

    # Test dataset evaluation using `Evaluator`
    model_predictions = [
        generate_language_model_output(input, model, tokenizer, alpaca_prompt_pl)
        for input in tqdm(test_list_pred)
    ]
    evaluate_df_metrics, avg_metrics = evaluate_language_model(
        model_predictions, test_list_target
    )
    mlflow.log_metrics(avg_metrics)

2025/05/06 09:04:43 INFO mlflow.tracking.fluent: Experiment with name 'PLLuM unsloth.ai' does not exist. Creating a new experiment.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,450 | Num Epochs = 1 | Total steps = 306
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,0.508200,0.524207
200,0.477100,0.487708
300,0.407900,0.473376


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


  0%|          | 0/150 [00:00<?, ?it/s]

Computing ROUGE scores:   0%|          | 0/150 [00:00<?, ?it/s]

Computing BERT scores:   0%|          | 0/150 [00:00<?, ?it/s]

🏃 View run PLLuM-full-epoch-v1 at: https://dagshub.com/informal2formal/mlflow.mlflow/#/experiments/4/runs/6741faad65cd47718e12a4e26e356949
🧪 View experiment at: https://dagshub.com/informal2formal/mlflow.mlflow/#/experiments/4


## Evaluate the model

### Example inputs

In [38]:
example_inputs = [
    "Mógłbyś mi przypomnieć, jaki był deadline na oddanie tego raportu? Coś mi świta, że to było w tym tygodniu.",
    "Trzeba opierdolić tych z supportu, bo czekam na odpowiedź już trzeci dzień, a problem dalej nierozwiązany.",
    "Pojebało cię człowieku żeby tam iść samemu!?",
    "Ta cała moda na 'self-care' i 'dbanie o siebie' jest ważna, ale czasem mam wrażenie, że przeradza się w egoizm i usprawiedliwienie dla olewania obowiązków albo innych ludzi. Granica jest cienka.",
    "Ten nowy pracownik w zespole to jakiś leser, ciągle łazi na fajkę albo siedzi na TikToku zamiast robić."
]
model_outputs = []

for input in tqdm(example_inputs):
    generated_output = generate_language_model_output(input, model, tokenizer, alpaca_prompt_pl)
    model_outputs.append(generated_output)

    print(generated_output, "\n")

  0%|          | 0/5 [00:00<?, ?it/s]

Czy mógłbyś mi przypomnieć, jaki był termin oddania tego raportu? Wydaje mi się, że upływał on w tym tygodniu. 

Należy zwrócić uwagę pracownikom działu wsparcia technicznego, ponieważ od trzech dni nie otrzymałem/otrzymałam odpowiedzi na zgłoszenie, a problem nadal nie został rozwiązany. 

Dlaczego postanowił Pan/Pani tam pójść samodzielnie? 

Moda na'self-care' i 'dbanie o siebie' jest zjawiskiem pozytywnym, jednakże należy pamiętać, że może ona przerodzić się w egoizm i usprawiedliwienie dla unikania obowiązków lub ignorowania potrzeb innych osób; cienka jest granica między dbaniem o siebie a nadmiernym skupieniem na sobie. 

Nowy pracownik w naszym zespole wykazuje niską motywację do pracy, często spędzając czas na paleniu papierosów lub korzystaniu z telefonu komórkowego (np. TikTok) zamiast wykonywać obowiązki. 



In [39]:
pd.set_option('display.max_colwidth', 300)
pd.DataFrame({
    "input": example_inputs,
    "base_model_output": base_model_outputs,
    "model_output": model_outputs,
})

,input,base_model_output,model_output
0,"Mógłbyś mi przypomnieć, jaki był deadline na oddanie tego raportu? Coś mi świta, że to było w tym tygodniu.","Przypominam, że termin złożenia raportu upływał w tym tygodniu.","Czy mógłbyś mi przypomnieć, jaki był termin oddania tego raportu? Wydaje mi się, że upływał on w tym tygodniu."
1,"Trzeba opierdolić tych z supportu, bo czekam na odpowiedź już trzeci dzień, a problem dalej nierozwiązany.","Należy wystosować do supportu stanowczy monit w sprawie przyspieszenia procedowania zgłoszonego wniosku, ponieważ od momentu złożenia wniosek nie został jeszcze rozpatrzony.","Należy zwrócić uwagę pracownikom działu wsparcia technicznego, ponieważ od trzech dni nie otrzymałem/otrzymałam odpowiedzi na zgłoszenie, a problem nadal nie został rozwiązany."
2,Pojebało cię człowieku żeby tam iść samemu!?,"Niech Pan wybaczy, ale nie rozumiem powodu Pana oburzenia.",Dlaczego postanowił Pan/Pani tam pójść samodzielnie?
3,"Ta cała moda na 'self-care' i 'dbanie o siebie' jest ważna, ale czasem mam wrażenie, że przeradza się w egoizm i usprawiedliwienie dla olewania obowiązków albo innych ludzi. Granica jest cienka.","Przedstawiamy tłumaczenie na język formalny:\n\nModa na ""samopielęgnację"" i ""dbanie o siebie"", która obecnie panuje, jest niewątpliwie istotna, jednakże niekiedy odnoszę wrażenie, że przekształca się ona w egoizm oraz usprawiedliwienie dla zaniedbywania obowiązków bądź innych osób. Granica jest ...","Moda na'self-care' i 'dbanie o siebie' jest zjawiskiem pozytywnym, jednakże należy pamiętać, że może ona przerodzić się w egoizm i usprawiedliwienie dla unikania obowiązków lub ignorowania potrzeb innych osób; cienka jest granica między dbaniem o siebie a nadmiernym skupieniem na sobie."
4,"Ten nowy pracownik w zespole to jakiś leser, ciągle łazi na fajkę albo siedzi na TikToku zamiast robić.","Należy stwierdzić, że nowy pracownik nie wywiązuje się należycie z powierzonych mu obowiązków, gdyż zamiast wykonywać powierzone mu zadania, zajmuje się niepotrzebnymi czynnościami, takimi jak palenie papierosów czy korzystanie z portali społecznościowych.","Nowy pracownik w naszym zespole wykazuje niską motywację do pracy, często spędzając czas na paleniu papierosów lub korzystaniu z telefonu komórkowego (np. TikTok) zamiast wykonywać obowiązki."


### Test dataset using `Evaluator`

In [40]:
avg_metrics

{'rouge1': 0.4566231206164584,
 'rouge2': 0.26061697783827587,
 'rougeL': 0.41311927058907166,
 'bert_precision': 0.8153831152121226,
 'bert_recall': 0.8055092052618662,
 'bert_f1': 0.8101584164301554}

In [41]:
evaluate_df_metrics.sort_values(by='bert_f1')

,pred,target,rouge1,rouge2,rougeL,bert_precision,bert_recall,bert_f1
57,Takie życie jest dla mnie nie do zniesienia.,Wyrażam głębokie niezadowolenie ze swojego życia.,0.000000,0.000000,0.000000,0.712078,0.687115,0.699374
50,Co za dziwny dzień.,To był wyjątkowo trudny/nieprzyjemny dzień.,0.181818,0.000000,0.181818,0.734328,0.678494,0.705308
56,"Ten parlamentarzysta zachowuje się w sposób niegodny, pozbawiony kultury politycznej.",Styl bycia i wypowiedzi niektórych parlamentarzystów są oceniane jako nieodpowiadające powadze pełnionej funkcji i standardom kultury osobistej.,0.125000,0.000000,0.125000,0.736264,0.691674,0.713273
97,"Nagromadzenie zaległości technologicznych w naszym projekcie jest obecnie na takim poziomie, że konieczne jest podjęcie natychmiastowych działań w celu nadrobienia zaległości.","Obserwujemy niepokojąco szybki wzrost długu technologicznego w naszym projekcie. Konieczne jest zaplanowanie i rozpoczęcie działań mających na celu jego stopniową redukcję (refaktoryzację, aktualizację bibliotek itp.).",0.350877,0.145455,0.315789,0.734558,0.694228,0.713824
59,"Rząd podejmuje działania, które mogą być postrzegane jako próba walki z inflacją, jednak ich rzeczywisty wpływ na jej ograniczenie jest kwestionowany.","Polityka gospodarcza rządu jest krytykowana za rzekome przyczynianie się do wzrostu inflacji, pomimo deklaracji o jej zwalczaniu.",0.136364,0.000000,0.090909,0.729164,0.706772,0.717793
...,...,...,...,...,...,...,...,...
34,Muszę udać się do sklepu po bułki.,Muszę udać się do sklepu po pieczywo.,0.800000,0.769231,0.800000,0.956272,0.940169,0.948152
87,Jestem zmęczony/zmęczona ciągłymi zmianami.,Jestem zmęczony/zmęczona tymi ciągłymi zmianami.,0.947368,0.823529,0.947368,0.972874,0.958507,0.965637
12,Należałoby odświeżyć znajomość języka angielskiego.,Należałoby odświeżyć swoją znajomość języka angielskiego.,0.952381,0.842105,0.952381,0.982870,0.971738,0.977272
127,Musimy przeanalizować naszą ofertę.,Musimy ponownie przeanalizować naszą ofertę.,0.888889,0.571429,0.888889,0.986230,0.972288,0.979209


## Save & upload the model

In [42]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
model.save_pretrained(run_name)
tokenizer.save_pretrained(run_name)

('PLLuM-1-full-epoch/tokenizer_config.json',
 'PLLuM-1-full-epoch/special_tokens_map.json',
 'PLLuM-1-full-epoch/tokenizer.json')

In [45]:
!cp -r PLLuM-1-full-epoch/ drive/MyDrive/Colab\ Notebooks

In [ ]:
# model.push_to_hub("PLLuM-v2-150-steps")
# tokenizer.push_to_hub("PLLuM-v2-150-steps")